In [1]:
import os

subj = "Subj1"
N_STAGES = 9
RERUN = False
exp = "exp_select_features"
os.makedirs(f"{subj}/{exp}", exist_ok = True)

In [2]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import SDA
import SDA.analytics
import SDA.clustquality

import umap
import tqdm
import pandas
import sklearn.metrics
import sklearn.preprocessing
import sklearn.decomposition
import tqdm.contrib.itertools
import sklearn.feature_selection
import sklearn.cross_decomposition

In [3]:
def explained_variance(features, reduced):
    pls = sklearn.cross_decomposition.PLSRegression(n_components = reduced.shape[1])
    y_pred = pls.fit(reduced, features).predict(reduced)
    return sklearn.metrics.r2_score(features, y_pred, multioutput = "variance_weighted")

### Selection

In [4]:
df_features = pandas.read_feather(f'{subj}/exp_final_filtered/all_features.feather')
print(df_features.shape)

features = sklearn.preprocessing.StandardScaler().fit_transform(df_features)
print(features.shape)

N_STAGES = 5
NUM_FEATURES = 765

params = {
    'n_clusters_min': 8, 'n_clusters_max': 8,
    'k_neighbours_min': 20, 'k_neighbours_max': 35,
    'len_st_thr': [ 20 ],

    'n_cl_max_thr': [ 8 ],
    'k_neighb_max_thr': [ 35 ],
    'n_edge_clusters_min': N_STAGES - 1, 'n_edge_clusters_max': N_STAGES - 1
}

(1046, 3799)
(1046, 3799)


In [5]:
if RERUN:
    scores = [ ]
    for i in tqdm.trange(features.shape[1]):
        try:
            result, _ = SDA.SDA(**params, n_jobs = 15, scale = False, verbose = False).apply(features[:, i].reshape(-1, 1))
            result = SDA.analytics.best_result(result, key = 'Avg-Silh', n_stages = N_STAGES)
            if len(result['St_edges']) != N_STAGES + 1:
                raise RuntimeError()
            score = result['Avg-Silh']
        except:
            score = -1
        scores.append({ 'index': i, 'name': df_features.columns[i], 'score': score })
    scores = pandas.DataFrame(scores)
else:
    scores = pandas.read_csv(f"{subj}/{exp}/scores.csv")
display(scores)

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,index,name,score
0,0,0,0,0,0,channel-0 entropy dim-1,0.160363
1,1,1,1,1,1,channel-0 entropy dim-2,0.132280
2,2,2,2,2,2,channel-0 numberofpoints dim-1,0.182975
3,3,3,3,3,3,channel-0 numberofpoints dim-2,0.183828
4,4,4,4,4,4,channel-0 amplitude-bottleneck dim-1,0.472430
...,...,...,...,...,...,...,...
3794,3794,3794,3794,3794,3794,overall bd2 dim-3 mean,0.075003
3795,3795,3795,3795,3795,3795,overall bd2 dim-3 std,0.023684
3796,3796,3796,3796,3796,3796,overall bd2 dim-3 sum,0.155738
3797,3797,3797,3797,3797,3797,overall bd2 dim-3 norm-1,0.155738


In [6]:
scores.to_csv(f"{subj}/{exp}/scores.csv")
display(scores.sort_values(by = 'score', ascending = False))

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,index,name,score
2437,2437,2437,2437,2437,2437,channel-26 amplitude-silhouette-1-2 dim-2,0.578132
2053,2053,2053,2053,2053,2053,channel-22 amplitude-landscape-1-2 dim-2,0.558201
2441,2441,2441,2441,2441,2441,channel-26 amplitude-silhouette-2-1 dim-2,0.533642
2347,2347,2347,2347,2347,2347,channel-25 amplitude-silhouette-1-2 norm-2,0.506922
2324,2324,2324,2324,2324,2324,channel-25 amplitude-landscape-1-1 dim-1,0.490045
...,...,...,...,...,...,...,...
2346,2346,2346,2346,2346,2346,channel-25 amplitude-silhouette-1-2 norm-1,-1.000000
1959,1959,1959,1959,1959,1959,channel-21 amplitude-landscape-1-1 norm-2,-1.000000
89,89,89,89,89,89,channel-0 bd2 dim-2 sum,-1.000000
1144,1144,1144,1144,1144,1144,channel-12 amplitude-silhouette-1-1 dim-1,-1.000000


In [7]:
best_feature_idx = scores.sort_values(by = 'score', ascending = False)[:NUM_FEATURES]["index"].to_numpy()
best_features = features[:, best_feature_idx]
print(best_features.shape)

(1046, 765)


### UMAP

In [8]:
umap = umap.UMAP(n_components = 15, random_state = 42)
umap_features = umap.fit_transform(best_features)

print(umap_features.shape)
print(explained_variance(best_features, umap_features))

(1046, 15)
0.5302920828684512


In [9]:
result, _ = SDA.SDA(n_jobs = 15, scale = False, verbose = True).apply(umap_features)
display(SDA.analytics.best_results(result, key = 'Avg-Silh'))

Applying to 1046 samples with 15 features each
Running stage 1


  0%|          | 0/589 [00:00<?, ?it/s]

Running stage 2


  0%|          | 0/672 [00:00<?, ?it/s]

,St_len_min,K_nb_max,N_cl_max,N_stages,Cl_cen,St_edges,Ward_dist,Cen_dist,Silh,Cal-Har,Dav-Bold,Avg-Silh,Avg-Cal-Har,Avg-Dav-Bold
0,0,35,10,3,Mode,"[0, 290, 682, 1046]",3161.919283,3.474061,0.215219,860.349035,3.395348,0.333131,850.619175,2.663925
1,0,35,15,4,Mode,"[0, 290, 682, 787, 1046]",2573.245531,4.112788,0.200933,755.632592,5.558532,0.404413,700.134729,0.755719
2,0,50,10,5,Mode,"[0, 290, 554, 682, 787, 1046]",1791.529439,4.181176,0.225177,801.903369,1.843722,0.515889,779.809101,0.661098
3,0,35,20,6,Median,"[0, 280, 556, 682, 795, 976, 1046]",1546.590371,3.872199,0.207794,738.661037,2.906555,0.468554,683.577151,0.827008
4,0,35,20,7,Median,"[0, 280, 556, 682, 787, 856, 976, 1046]",1258.147809,3.647017,0.195297,734.306928,3.642418,0.500515,592.476122,0.739815
5,60,40,20,8,Median,"[0, 190, 290, 556, 682, 787, 856, 976, 1046]",688.150690,3.224527,0.113321,703.074177,3.236455,0.468854,347.111136,0.814997
6,0,35,15,9,Mode,"[0, 55, 244, 290, 556, 682, 787, 856, 976, 1046]",495.462799,2.917004,0.066165,643.867856,3.057780,0.433679,262.338960,0.908948
7,60,45,15,10,Median,"[0, 199, 290, 509, 556, 664, 682, 787, 856, 97...",317.153436,2.645310,0.122616,580.820142,3.814688,0.414563,166.750233,1.074652
8,20,45,15,11,Median,"[0, 55, 244, 290, 509, 556, 665, 682, 787, 856...",221.374642,2.459239,0.066885,544.435407,3.727175,0.387424,122.004968,1.138406
9,20,45,15,12,Median,"[0, 55, 190, 244, 290, 509, 556, 665, 682, 787...",189.069766,2.259472,0.058645,510.487643,3.566863,0.362128,105.185977,1.195828


### PCA

In [10]:
pca = sklearn.decomposition.PCA(n_components = 15, svd_solver = "full", random_state = 42)
pca_features = pca.fit_transform(best_features)

print('Explained variance', round(pca.explained_variance_ratio_.sum(), 2))
print([ round(x, 3) for x in pca.explained_variance_ratio_ ])
print(pca_features.shape)
print(explained_variance(best_features, pca_features))

Explained variance 0.75
[0.294, 0.111, 0.083, 0.051, 0.034, 0.03, 0.023, 0.023, 0.018, 0.018, 0.015, 0.013, 0.013, 0.013, 0.012]
(1046, 15)
0.7509675089020927


In [11]:
result, _ = SDA.SDA(n_jobs = 15, scale = False, verbose = True).apply(pca_features)
display(SDA.analytics.best_results(result, key = 'Avg-Silh'))

Applying to 1046 samples with 15 features each
Running stage 1


  0%|          | 0/589 [00:00<?, ?it/s]

Running stage 2


  0%|          | 0/672 [00:00<?, ?it/s]

,St_len_min,K_nb_max,N_cl_max,N_stages,Cl_cen,St_edges,Ward_dist,Cen_dist,Silh,Cal-Har,Dav-Bold,Avg-Silh,Avg-Cal-Har,Avg-Dav-Bold
0,60,40,15,3,Mode,"[0, 210, 556, 1046]",42015.483009,15.762431,0.061632,136.386109,3.418155,0.155654,109.947265,2.930341
1,40,40,20,4,Mode,"[0, 210, 682, 795, 1046]",58313.368541,22.714365,0.050441,120.938596,4.623428,0.209465,135.506905,1.555254
2,40,50,15,5,Mode,"[0, 210, 556, 682, 795, 1046]",57059.170329,25.675386,0.068703,137.352410,2.827397,0.288836,155.834511,1.383560
3,60,40,20,6,Mode,"[0, 210, 556, 682, 856, 976, 1046]",52141.731206,25.177458,0.054235,124.617161,3.210948,0.295695,154.401042,1.331537
4,20,45,20,7,Mode,"[0, 55, 210, 556, 682, 795, 976, 1046]",40594.731614,22.710237,0.042044,106.395345,3.105567,0.254761,117.979753,1.629336
5,60,45,20,8,Median,"[0, 210, 310, 556, 682, 789, 856, 976, 1046]",27577.629798,20.732497,0.022504,102.786262,3.583614,0.239136,85.531844,1.655676
6,40,45,20,9,Median,"[0, 55, 210, 310, 556, 682, 789, 856, 976, 1046]",24828.727764,19.856665,0.019410,94.780000,3.517684,0.224941,76.753998,1.754172
7,60,40,20,10,Median,"[0, 194, 210, 250, 313, 556, 682, 795, 856, 97...",19524.170155,18.335139,-0.016890,82.094757,3.605503,0.216702,62.023352,1.901630
8,0,40,20,11,Mode,"[0, 55, 141, 210, 313, 556, 659, 795, 856, 976...",18315.209353,18.748918,-0.000040,73.127844,3.779201,0.213725,55.651415,2.141425
9,0,45,20,12,Mode,"[0, 55, 141, 210, 313, 489, 556, 682, 795, 856...",15092.483590,17.662368,-0.006272,72.756745,3.771718,0.191680,47.703967,2.198586
